In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%run figures.py 'new_mins_30-01-24.nc' -c volume
#%run figures.py 'original_mins_low_res_19-01-24.nc' -c volume


In [2]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
import copy
import coeus.plots as plots
import string

path_15 = home / 'work' / 'data' / 'hydrothermal_alteration' / 'long_runs' /'hta_15deg.nc'
data_15 = {}
data_15.update({'volume': xr.open_dataset(path_15, group='volume')})

time_steps = 25
i = 0
file_num=0
fontsize = 24
time = 21

fig, axes = plt.subplots(1,1, figsize=(11,9))
axis = axes

category = 'volume'
plot_vars = ['Anhydrite'] * 4

leg_entries = ['SO$\mathbf{_4^{2-}}$', 'HCO$\mathbf{_3^-}$', 'Ca$\mathbf{^{2+}}$', 'Mg$\mathbf{^{2+}}$']
scale_factors = (1,1,1,1)
range_set1 = [(10,13), (14,18), (6,9), (0,5)]
labels1 = [[10, 20, 30, 40],
          [2.0, 2.2, 2.4, 2.6, 2.8],
          [10, 20, 30, 40],
          [10, 20, 30, 40, 50, 60]]
range_set2 = [(10,14), (15,19), (6,9), (0,5)]
labels2 = [[1, 10, 20, 30, 40],
           [2.0, 2.2, 2.4, 2.6, 2.8],
           [10, 20, 30, 40],
           [10, 20, 30, 40, 50, 60]]

range_set_list = [range_set1, range_set1]
label_list = [labels1, labels1]

linestyles = ['-', '--']

colours = ['tab:green', 'tab:red', 'tab:orange', 'tab:blue']

temperatures = [2, 15]

legend_titles=[1,2,3,4]

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
anhyd_mv = 46.010
vf_to_mols = cell_vol / anhyd_mv

scale_factors = (vf_to_mols, vf_to_mols, vf_to_mols, vf_to_mols)

scaling = 'global'

# Format axis
axins1 = axis.inset_axes([0.58, 0.08, 0.37, 0.37])
plots.format_axis(axins1, font_props)
plots.format_axis(axis, font_props)
axis.set_xlabel('Concentration / mM')
if scaling == 'global':
    axis.set_ylabel('Global S flux into Crust / Tmol yr$^{-1}$')
    #axis.set_ylim(0.19,1)
    axis.set_ylim(0,1)
    axis.set_xlim(0,65)
    axins1.set_ylim(0.19,1)
    axins1.set_xlim(1.9,2.9)
else:
    axis.set_ylabel('Anhydrite formed in column / mol')
    axis.set_ylim(80000,355000)
    axis.set_xlim(0,65)
    axins1.set_ylim(80000,355000)
    axins1.set_xlim(1.9,2.9)

axins1.xaxis.set_major_locator(plt.MaxNLocator(3))
axins1.yaxis.set_major_locator(plt.MaxNLocator(5))

legend_props = {'weight': 'bold', 'size':18}

datasets = [data, data_15]

for dataset, ls, temperature, labels, range_sets in zip(datasets, linestyles, temperatures, label_list, range_set_list):

    if temperature == 15:
        leg_entries = ['_nolegend_'] * 4

    iterables = zip(np.arange(len(plot_vars)), plot_vars, legend_titles, scale_factors, range_sets, labels, leg_entries, colours)
    # Axis level loop
    for j, plot_var, legend_title, scale, range_set, label_a, leg_entry, col in iterables:
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0], range_set[1]+1)
        y = []
        x = copy.deepcopy(label_a)
        pop_list = []

        for n, ind in enumerate(index):
            try:
                val = dataset[category].isel(file_num = ind, time=time, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
                if val == 0:
                    pop_list.append(n)
                else:
                    y.append(val)
            except:
                pop_list.append(n)
                continue

        pop_list.sort(reverse=True)
        for q in pop_list:
            x.pop(q)

        if j == 1:
            if scaling == 'global':
                axins1.plot(x, s_moles_to_flux(y), label=f'{leg_entry}', linestyle=ls, c=col, linewidth=7)
            else:
                axins1.plot(x, y, label=f'{leg_entry}', linestyle=ls, c=col, linewidth=7)
            axis.plot(np.NaN, np.NaN, label=f'{leg_entry}', c=col, ls = ls, linewidth=7)
        else:
            if scaling == 'global':
                axis.plot(x, s_moles_to_flux(y), label= f'{leg_entry}', linestyle=ls, c=col, linewidth=7)
            else:
                axis.plot(x, y, label= f'{leg_entry}', linestyle=ls, c=col, linewidth=7)
        # Axis decorations
        axis.legend(title='Seawater component', title_fontproperties=legend_props, prop=legend_props, fontsize=12, framealpha=0, loc=2)
        fig.tight_layout()
        fig.savefig(f'output/{j}_{temperature}.png', dpi=300)


styles = ['-', '--']
ax2 = axis.twinx()
for sty, temp in zip(styles, temperatures):
    ax2.plot(np.NaN, np.NaN, ls=sty,
             label=temp, c='black')
ax2.get_yaxis().set_visible(False)
ax2.legend(title='Temperature / °C', prop=font_props, title_fontproperties=font_props, framealpha=0, loc=1, linewidth=7)

fig.show()
fig.savefig('output/anhydrite_v_cations.png', dpi=300, transparent=True)

